# NOAA Analysis

In [2]:
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.3f' % x)
%matplotlib inline

In [41]:
event_details = pd.read_csv("results/combined_0.csv");
event_details.shape

/Users/dmo/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (16,25,26,28,29,34,35,37,39,40,42,43,48,49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(1457423, 51)

In [42]:
event_details.head(3)

,BEGIN_YEARMONTH,BEGIN_DAY,BEGIN_TIME,END_YEARMONTH,END_DAY,END_TIME,EPISODE_ID,EVENT_ID,STATE,STATE_FIPS,...,END_RANGE,END_AZIMUTH,END_LOCATION,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,EPISODE_NARRATIVE,EVENT_NARRATIVE,DATA_SOURCE
0,195004,28,1445,195004,28,1445,nan,10096222,OKLAHOMA,40.000,...,0.000,NaN,NaN,35.120,-99.200,35.170,-99.200,NaN,NaN,PUB
1,195004,29,1530,195004,29,1530,nan,10120412,TEXAS,48.000,...,0.000,NaN,NaN,31.900,-98.600,31.730,-98.600,NaN,NaN,PUB
2,195007,5,1800,195007,5,1800,nan,10104927,PENNSYLVANIA,42.000,...,0.000,NaN,NaN,40.580,-75.700,40.650,-75.470,NaN,NaN,PUB


### Problem 1 - summarize the property damage, by state, by year

In [48]:
event_details[(event_details['STATE'] == 'NEW JERSEY') & (event_details['YEAR'] == 2012)][['EVENT_ID','DAMAGE_PROPERTY']].head()

,EVENT_ID,DAMAGE_PROPERTY
1118865,367509,0.00K
1120962,367807,0.00K
1129720,413112,900.00K
1129900,420880,0.00K
1129901,420881,0.00K


The property damage column is not in number form. A quick glance shows that this field is suffixed with K representing thousand and M representing million. Next, we will convert this into number form.

In [44]:
# stolen from https://stackoverflow.com/questions/39684548/convert-the-string-2-90k-to-2900-or-5-2m-to-5200000-in-pandas-dataframe
def value_to_float(x):
    x = str(x)
    if 'K' in x:
        if len(x) > 1:
            return float(x.replace('K', '')) * 1000
        return 1000.0
    if 'M' in x:
        if len(x) > 1:
            return float(x.replace('M', '')) * 1000000
        return 1000000.0
    if 'B' in x:
        return float(x.replace('B', '')) * 1000000000
    return 0.0

In [49]:
event_details['DAMAGE_PROPERTY_DOLLARS'] = event_details['DAMAGE_PROPERTY'].apply(value_to_float)

In [50]:
# sanity check - make sure data is being converted as expected
#nj_damages = event_details[event_details['STATE'] == 'NEW JERSEY']
#nj_damages[nj_damages['YEAR'] == 2012][['EVENT_ID','DAMAGE_PROPERTY_DOLLARS','DAMAGE_PROPERTY']].head(98)
#nj_damages[['EVENT_ID','DAMAGE_PROPERTY_DOLLARS','DAMAGE_PROPERTY','YEAR']].to_csv("nj_damages.csv")

In [51]:
event_details['STATE'].unique()

array(['OKLAHOMA', 'TEXAS', 'PENNSYLVANIA', 'NEBRASKA', 'MISSISSIPPI',
       'NEW MEXICO', 'ARKANSAS', 'MISSOURI', 'CONNECTICUT', 'FLORIDA',
       'NORTH CAROLINA', 'ALABAMA', 'KENTUCKY', 'MARYLAND', 'MINNESOTA',
       'SOUTH DAKOTA', 'IOWA', 'LOUISIANA', 'OHIO', 'KANSAS',
       'NORTH DAKOTA', 'INDIANA', 'COLORADO', 'SOUTH CAROLINA',
       'WEST VIRGINIA', 'WYOMING', 'GEORGIA', 'WISCONSIN', 'ILLINOIS',
       'TENNESSEE', 'NEW JERSEY', 'MICHIGAN', 'CALIFORNIA',
       'MASSACHUSETTS', 'NEW HAMPSHIRE', 'OREGON', 'VIRGINIA', 'ARIZONA',
       'NEW YORK', 'MONTANA', 'MAINE', 'VERMONT', 'UTAH', 'DELAWARE',
       'IDAHO', 'WASHINGTON', 'HAWAII', 'RHODE ISLAND', 'NEVADA',
       'PUERTO RICO', 'DISTRICT OF COLUMBIA', 'ALASKA', 'VIRGIN ISLANDS',
       'GUAM', 'AMERICAN SAMOA', 'ATLANTIC SOUTH', 'LAKE ERIE',
       'LAKE HURON', 'GULF OF MEXICO', 'LAKE ST CLAIR', 'E PACIFIC',
       'HAWAII WATERS', 'LAKE MICHIGAN', 'ATLANTIC NORTH', 'LAKE ONTARIO',
       'ST LAWRENCE R', 'LAKE SUPERI

We see that there are non-states in our dataset, we need to be sure to filter these out.

In [52]:
states = pd.read_csv("states.csv")
#upper all states so they match file
states['State'] = states['State'].apply(lambda x: x.upper())
event_details_state = event_details[event_details['STATE'].isin(states['State'])].copy()
event_details_state.shape

(1424943, 52)

We went from 1,457,423 to 1,424,943 after filtering non US states.

In [53]:
event_details_grouped = event_details_state.groupby(['STATE','YEAR'])['DAMAGE_PROPERTY_DOLLARS'].sum()

In [54]:
event_details_grouped.head()

STATE    YEAR
ALABAMA  1950     27500.000
         1951     35000.000
         1952   5452500.000
         1953   3070000.000
         1954    607530.000
Name: DAMAGE_PROPERTY_DOLLARS, dtype: float64

In [55]:
# outputs to csv file
event_details_grouped.to_csv("property_damage_summary.csv")

### Problem 2 - where to build to minimize property damage

In [56]:
event_details_grouped = event_details_state.groupby(['STATE'])['DAMAGE_PROPERTY_DOLLARS'].sum().sort_values(ascending=False)
event_details_grouped.head()

STATE
LOUISIANA     68120714540.000
TEXAS         47104718850.000
FLORIDA       41441290320.000
MISSISSIPPI   31003482190.000
NEW JERSEY    27795146590.000
Name: DAMAGE_PROPERTY_DOLLARS, dtype: float64

In terms of dollars, Louisiana is clearly in the worst shape due to disasters with over 68 billion in damages, a good portion of that due to Katrina. Unforunately, New Jersey is also in the top 5 with almost almost 28 billion in damages, which mostly can be attributed to Sandy. 

In [57]:
state_land_area = pd.read_csv('states_landarea.csv')
state_land_area.set_index('STATE', inplace=True,)
# convert to dict for constant time access
state_land_area_dict = state_land_area.to_dict()['AREA']

def calc_damage_per_sqmi(state_row):
    try:
        
        land_area = int(state_land_area_dict[state_row['STATE']].replace(',', ''))
        damage_dollars = state_row['DAMAGE_PROPERTY_DOLLARS']
        return damage_dollars/land_area
    except:
        print("Could not locate state {} in land area file".format(state))
        return None

In [58]:
event_details_state['DAMAGE_PROPERTY_DOLLARS_SQMI'] = event_details_state[['DAMAGE_PROPERTY_DOLLARS','STATE']].apply(calc_damage_per_sqmi,axis=1);
event_details_state.head()

,BEGIN_YEARMONTH,BEGIN_DAY,BEGIN_TIME,END_YEARMONTH,END_DAY,END_TIME,EPISODE_ID,EVENT_ID,STATE,STATE_FIPS,...,END_LOCATION,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,EPISODE_NARRATIVE,EVENT_NARRATIVE,DATA_SOURCE,DAMAGE_PROPERTY_DOLLARS,DAMAGE_PROPERTY_DOLLARS_SQMI
0,195004,28,1445,195004,28,1445,nan,10096222,OKLAHOMA,40.000,...,NaN,35.120,-99.200,35.170,-99.200,NaN,NaN,PUB,250000.000,3.577
1,195004,29,1530,195004,29,1530,nan,10120412,TEXAS,48.000,...,NaN,31.900,-98.600,31.730,-98.600,NaN,NaN,PUB,25000.000,0.093
2,195007,5,1800,195007,5,1800,nan,10104927,PENNSYLVANIA,42.000,...,NaN,40.580,-75.700,40.650,-75.470,NaN,NaN,PUB,25000.000,0.543
3,195007,5,1830,195007,5,1830,nan,10104928,PENNSYLVANIA,42.000,...,NaN,40.600,-76.750,nan,nan,NaN,NaN,PUB,2500.000,0.054
4,195007,24,1440,195007,24,1440,nan,10104929,PENNSYLVANIA,42.000,...,NaN,41.630,-79.680,nan,nan,NaN,NaN,PUB,2500.000,0.054


In [59]:
event_details_state_grouped = event_details_state.groupby(['STATE'])[['DAMAGE_PROPERTY_DOLLARS_SQMI','DAMAGE_PROPERTY_DOLLARS']].sum()

In [ ]:
# grouped by state and year
#event_details_state_grouped = event_details_state.groupby(['STATE','YEAR'])[['DAMAGE_PROPERTY_DOLLARS_SQMI','DAMAGE_PROPERTY_DOLLARS']].sum()

#### Which state to build a house in to minimize risk?
Assuming you were to randomly pick a spot in a state to build (and able to do so), New Jersey would be one of the worst places to do so, with 3.18m per square mile in total historical damages. Alaska would be the best, with 544 per square mile. Data broken down by states is shown below.

However, these numbers are likely thrown off by high density areas and the cost of living for the area. Alaska has a lot more unoccupied territory than New Jersey. 

If we wanted to truly know which areas are best to build a house in, we would use the coordinates of the disaster data to build a map that shows the area that is affected, colored by how much damage it caused. Multiple events ontop of eachother would cause the color to be more extreme.


In [60]:
event_details_state_grouped.sort_values(by = 'DAMAGE_PROPERTY_DOLLARS_SQMI',ascending=False, inplace=True)
event_details_state_grouped.head(55)

,DAMAGE_PROPERTY_DOLLARS_SQMI,DAMAGE_PROPERTY_DOLLARS
STATE,,
NEW JERSEY,3186420.565,27795146590.000
DISTRICT OF COLUMBIA,2319089.706,157698100.000
LOUISIANA,1300559.673,68120714540.000
MISSISSIPPI,640144.578,31003482190.000
FLORIDA,630209.105,41441290320.000
ALABAMA,229473.829,12029018120.000
NORTH CAROLINA,176403.580,9493864260.000
TEXAS,175373.866,47104718850.000
OHIO,167318.760,7500230740.000
